In [1]:
class Iterator(object):
    def __init__(self):
        self.count=0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if  self.count > 2:
            raise StopIteration()
            
        self.count += 1
        
        return self.count

In [2]:
iterator = Iterator()
iterator

In [3]:
for c in iterator:
    print('count:', c)

count: 1
count: 2
count: 3


In [4]:
import numpy as np
import pandas as pd
import urllib.request 
from rdkit import Chem
from rdkit.Chem import AllChem, Draw, Descriptors, PandasTools
from rdkit.ML.Descriptors import MoleculeDescriptors
from sklearn.model_selection import train_test_split

url = 'https://raw.githubusercontent.com/onecoinbuybus/Database_chemoinformatics/master/smiles_cas_N6512.smi'
urllib.request.urlretrieve(url, 'ames.txt') 
df = pd.read_csv('ames.txt',header=None, sep='\t') 
df.columns = ['smiles', 'CAS_NO', 'activity']
PandasTools.AddMoleculeColumnToFrame(frame=df, smilesCol='smiles')

none_list=[]
for i in range(df.shape[0]):
    if Chem.MolFromSmiles(df['smiles'][i]) is None:
        none_list.append(i)
        
df=df.drop(none_list)
mols=[Chem.MolFromSmiles(smile) for smile in df['smiles']]

maccskeys = []
for m in mols:
    maccskey = [x for x in AllChem.GetMACCSKeysFingerprint(m)]
    maccskeys.append(maccskey)
    
X = np.array(maccskeys)
y=df['activity'].values.reshape(-1, 1)
X.shape
#(6506, 167)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

RDKit ERROR: [14:15:49] SMILES Parse Error: syntax error while parsing: NNC(=O)CNC(=O)\C=N\#N
RDKit ERROR: [14:15:49] SMILES Parse Error: Failed parsing SMILES 'NNC(=O)CNC(=O)\C=N\#N' for input: 'NNC(=O)CNC(=O)\C=N\#N'
RDKit ERROR: [14:15:49] SMILES Parse Error: syntax error while parsing: O=C1NC(=O)\C(=N/#N)\C=N1
RDKit ERROR: [14:15:49] SMILES Parse Error: Failed parsing SMILES 'O=C1NC(=O)\C(=N/#N)\C=N1' for input: 'O=C1NC(=O)\C(=N/#N)\C=N1'
RDKit ERROR: [14:15:49] SMILES Parse Error: syntax error while parsing: NC(=O)CNC(=O)\C=N\#N
RDKit ERROR: [14:15:49] SMILES Parse Error: Failed parsing SMILES 'NC(=O)CNC(=O)\C=N\#N' for input: 'NC(=O)CNC(=O)\C=N\#N'
RDKit ERROR: [14:15:49] SMILES Parse Error: syntax error while parsing: CCCCN(CC(O)C1=C\C(=N/#N)\C(=O)C=C1)N=O
RDKit ERROR: [14:15:49] SMILES Parse Error: Failed parsing SMILES 'CCCCN(CC(O)C1=C\C(=N/#N)\C(=O)C=C1)N=O' for input: 'CCCCN(CC(O)C1=C\C(=N/#N)\C(=O)C=C1)N=O'
RDKit ERROR: [14:15:49] SMILES Parse Error: syntax error while pars

In [5]:
x_train.shape

(5204, 167)

In [6]:
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optimizers

np.random.seed(369)
torch.manual_seed(957)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.l1 = nn.Linear(input_dim, hidden_dim)
        self.a1 = nn.ReLU()
        self.l2 = nn.Linear(hidden_dim, output_dim)
        self.a2 = nn.ReLU()

        self.layers = [self.l1, self.a1, self.l2, self.a2]

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)

        return x
        return x

In [8]:
input_dim=x_train.shape[-1]
hiden_dim = 64
output_dim = 1

In [9]:
model = MLP(input_dim, hiden_dim, output_dim).to(device)
model

MLP(
  (l1): Linear(in_features=167, out_features=64, bias=True)
  (a1): ReLU()
  (l2): Linear(in_features=64, out_features=1, bias=True)
  (a2): ReLU()
)

In [10]:
criterion = nn.MSELoss()
optimizer = optimizers.Adam(model.parameters(),
                            lr=0.001,
                            betas=(0.9, 0.999), amsgrad=True)

In [11]:
from sklearn.utils import shuffle
import torch
#from tensorflow.keras.preprocessing.sequence import pad_sequences


class DataLoader(object):
    def __init__(self, dataset,
                 batch_size=100,
                 shuffle=False,
                 batch_first=False,
                 device='cpu',
                 random_state=None):
        self.dataset = list(zip(dataset[0], dataset[1]))
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.batch_first = batch_first
        self.device = device

        if random_state is None:
            random_state = np.random.RandomState(123)

        self.random_state = random_state
        self._idx = 0
        self._reset()

    def __len__(self):
        N = len(self.dataset)
        b = self.batch_size
        return N // b + bool(N % b)

    def __iter__(self):
        return self

    def __next__(self):
        if self._idx >= len(self.dataset):
            self._reset()
            raise StopIteration()

        x, t = zip(*self.dataset[self._idx:(self._idx + self.batch_size)])
        
#         x = pad_sequences(x, padding='post'), 需要时候加maxlen，从tf import
#         t = pad_sequences(t, padding='post')

        x = torch.Tensor(x).T #长sequence时候用LongTensor #tf用tf.convert_to_tensor(x, dtype=tf.float32)
        t = torch.Tensor(t).T

        if not self.batch_first:
            x = x.t()
            t = t.t()

        self._idx += self.batch_size

        return x.to(self.device), t.to(self.device)

    def _reset(self):
        if self.shuffle:
            self.dataset = shuffle(self.dataset,
                                   random_state=self.random_state)
        self._idx = 0


In [12]:
train_dataloader = DataLoader((x_train, y_train),batch_first=False,device=device)
val_dataloader = DataLoader((x_test, y_test),batch_first=False,device=device)

In [13]:
epochs = 5
def compute_loss(t, y):
        return criterion(y, t)

def train_step(x, t):
    model.train()
    preds = model(x)
    loss = compute_loss(t, preds)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss


In [14]:
for epoch in range(epochs):
    print('-' * 10, 'epoch: {}'.format(epoch+1),'-' * 10)
    train_loss = 0.
    for (x, y) in train_dataloader:
        loss = train_step(x, y)
        train_loss += loss.item()
    train_loss /= len(train_dataloader)
    print('loss: {:.3f}'.format(train_loss))
        

---------- epoch: 1 ----------
loss: 0.243
---------- epoch: 2 ----------
loss: 0.181
---------- epoch: 3 ----------
loss: 0.167
---------- epoch: 4 ----------
loss: 0.159
---------- epoch: 5 ----------
loss: 0.152
